In [1]:
from google.colab import files

In [2]:
import zipfile
zip_path="/content/drive/MyDrive/Dataset/Dataset.zip"
extract_path="/content/fish_data"

In [3]:
with zipfile.ZipFile(zip_path,'r') as zip_ref:
  zip_ref.extractall(extract_path)

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [5]:
train="/content/fish_data/images.cv_jzk6llhf18tm3k0kyttxz/data/train"
test="/content/fish_data/images.cv_jzk6llhf18tm3k0kyttxz/data/test"
val="/content/fish_data/images.cv_jzk6llhf18tm3k0kyttxz/data/val"

In [6]:
train_gen=ImageDataGenerator(rescale=1./255,rotation_range=20,zoom_range=0.2,horizontal_flip=True)
val_gen=ImageDataGenerator(rescale=1./255)

In [7]:
train_data=train_gen.flow_from_directory(train,target_size=(244,244),class_mode="categorical")
test_data=val_gen.flow_from_directory(test,target_size=(244,244),class_mode="categorical")
val_data=val_gen.flow_from_directory(val,target_size=(244,244),class_mode="categorical")

Found 6225 images belonging to 11 classes.
Found 3187 images belonging to 11 classes.
Found 1092 images belonging to 11 classes.


In [16]:
class_labels=train_data.class_indices.keys()
print(class_labels)

dict_keys(['animal fish', 'animal fish bass', 'fish sea_food black_sea_sprat', 'fish sea_food gilt_head_bream', 'fish sea_food hourse_mackerel', 'fish sea_food red_mullet', 'fish sea_food red_sea_bream', 'fish sea_food sea_bass', 'fish sea_food shrimp', 'fish sea_food striped_red_mullet', 'fish sea_food trout'])


In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,Dropout

In [9]:
model=Sequential([
    Conv2D(32,(3,3),activation='relu',input_shape=(244,244,3)),
    MaxPooling2D(2,2),
    Conv2D(64,(3,3),activation="relu"),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(64,activation="relu"),
    Dropout(0.3),
    Dense(11,activation="softmax")
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [10]:
model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])

In [11]:
model.fit(train_data,validation_data=val_data,epochs=10,batch_size=32)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
195/195 ━━━━━━━━━━━━━━━━━━━━ 107s 507ms/step - accuracy: 0.3042 - loss: 2.2111 - val_accuracy: 0.6300 - val_loss: 1.1252
Epoch 2/10
195/195 ━━━━━━━━━━━━━━━━━━━━ 93s 476ms/step - accuracy: 0.5869 - loss: 1.1843 - val_accuracy: 0.7555 - val_loss: 0.7669
Epoch 3/10
195/195 ━━━━━━━━━━━━━━━━━━━━ 147s 504ms/step - accuracy: 0.6961 - loss: 0.8815 - val_accuracy: 0.8800 - val_loss: 0.4554
Epoch 4/10
195/195 ━━━━━━━━━━━━━━━━━━━━ 92s 474ms/step - accuracy: 0.7510 - loss: 0.7210 - val_accuracy: 0.8626 - val_loss: 0.4600
Epoch 5/10
195/195 ━━━━━━━━━━━━━━━━━━━━ 92s 470ms/step - accuracy: 0.7764 - loss: 0.6203 - val_accuracy: 0.8846 - val_loss: 0.4240
Epoch 6/10
195/195 ━━━━━━━━━━━━━━━━━━━━ 92s 474ms/step - accuracy: 0.7827 - loss: 0.6408 - val_accuracy: 0.8956 - val_loss: 0.3363
Epoch 7/10
195/195 ━━━━━━━━━━━━━━━━━━━━ 94s 481ms/step - accuracy: 0.8242 - loss: 0.5137 - val_accuracy: 0.9432 - val_loss: 0.2206
Epoch 8/10
195/195 ━━━━━━━━━━━━━━━━━━━━ 94s 480ms/step - accuracy: 0.8439 - loss:

In [12]:
base_loss,base_accurracy=model.evaluate(test_data)

100/100 ━━━━━━━━━━━━━━━━━━━━ 7s 71ms/step - accuracy: 0.9430 - loss: 0.2060


In [29]:
model.save("fish_class_cnn.h5")
from google.colab import drive
drive.mount('/content/drive')
!cp fish_class_cnn.h5 /content/drive/MyDrive/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
from tensorflow.keras.applications import VGG16,ResNet50,InceptionV3,MobileNetV2,EfficientNetB0
from tensorflow.keras.layers import GlobalAveragePooling2D,Dense,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [21]:
def create_transfer_model(base_model):
  base_model.trainable=False

  x=base_model.output
  x=GlobalAveragePooling2D()(x)
  x=Dense(128,activation="relu")(x)
  x=Dropout(0.3)(x)
  predictions=Dense(11,activation="softmax")(x)

  transfer_model=Model(inputs=base_model.input,outputs=predictions)
  return transfer_model

In [22]:
vgg16_base=VGG16(weights="imagenet",include_top=False,input_shape=(244,244,3))
VGG16_model=create_transfer_model(vgg16_base)

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [23]:
VGG16_model.compile(optimizer=Adam(),loss="categorical_crossentropy",metrics=["accuracy"])

In [ ]:
VGG16_model.fit(train_data,validation_data=val_data,epochs=10,batch_size=100)

Epoch 1/10
195/195 ━━━━━━━━━━━━━━━━━━━━ 136s 698ms/step - accuracy: 0.5704 - loss: 1.4343 - val_accuracy: 0.8141 - val_loss: 0.8705
Epoch 2/10
195/195 ━━━━━━━━━━━━━━━━━━━━ 109s 560ms/step - accuracy: 0.7474 - loss: 0.8824 - val_accuracy: 0.8755 - val_loss: 0.5856
Epoch 3/10
195/195 ━━━━━━━━━━━━━━━━━━━━ 112s 572ms/step - accuracy: 0.8073 - loss: 0.6501 - val_accuracy: 0.8993 - val_loss: 0.4429
Epoch 4/10
195/195 ━━━━━━━━━━━━━━━━━━━━ 107s 550ms/step - accuracy: 0.8522 - loss: 0.4958 - val_accuracy: 0.9194 - val_loss: 0.3431
Epoch 5/10
195/195 ━━━━━━━━━━━━━━━━━━━━ 108s 553ms/step - accuracy: 0.8778 - loss: 0.4343 - val_accuracy: 0.9286 - val_loss: 0.3011
Epoch 6/10
195/195 ━━━━━━━━━━━━━━━━━━━━ 107s 548ms/step - accuracy: 0.8993 - loss: 0.3605 - val_accuracy: 0.9332 - val_loss: 0.2563
Epoch 7/10
195/195 ━━━━━━━━━━━━━━━━━━━━ 108s 556ms/step - accuracy: 0.9133 - loss: 0.3164 - val_accuracy: 0.9368 - val_loss: 0.2368
Epoch 8/10
195/195 ━━━━━━━━━━━━━━━━━━━━ 108s 554ms/step - accuracy: 0.9122 -

In [28]:
VGG16_loss,VGG16_accuracy=VGG16_model.evaluate(test_data)

100/100 ━━━━━━━━━━━━━━━━━━━━ 23s 224ms/step - accuracy: 0.9666 - loss: 0.1402


In [31]:
vgg16_base.trainable=True

VGG16_model.compile(optimizer=Adam(learning_rate=0.0001),loss="categorical_crossentropy",metrics=["accuracy"])

VGG16_model.fit(train_data,validation_data=val_data,epochs=5)

Epoch 1/5
195/195 ━━━━━━━━━━━━━━━━━━━━ 156s 760ms/step - accuracy: 0.9638 - loss: 0.1353 - val_accuracy: 0.9872 - val_loss: 0.0274
Epoch 2/5
195/195 ━━━━━━━━━━━━━━━━━━━━ 137s 698ms/step - accuracy: 0.9816 - loss: 0.0615 - val_accuracy: 0.9908 - val_loss: 0.0289
Epoch 3/5
195/195 ━━━━━━━━━━━━━━━━━━━━ 136s 698ms/step - accuracy: 0.9944 - loss: 0.0176 - val_accuracy: 0.9634 - val_loss: 0.1237
Epoch 4/5
195/195 ━━━━━━━━━━━━━━━━━━━━ 136s 698ms/step - accuracy: 0.9479 - loss: 0.1917 - val_accuracy: 0.9707 - val_loss: 0.0987
Epoch 5/5
195/195 ━━━━━━━━━━━━━━━━━━━━ 137s 699ms/step - accuracy: 0.9825 - loss: 0.0677 - val_accuracy: 0.9853 - val_loss: 0.0394


In [40]:
VGG16_model.save("VGG16_fish_class.h5")
from google.colab import drive
drive.mount('/content/drive')
!cp VGG16_fish_class.h5 /content/drive/MyDrive/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## ResNet50 Model

In [33]:
ResNet50_base=ResNet50(weights='imagenet',include_top=False,input_shape=(244,244,3))
ResNet50_model=create_transfer_model(ResNet50_base)

ResNet50_model.compile(optimizer=Adam(),loss="categorical_crossentropy",metrics=["accuracy"])

ResNet50_model.fit(train_data,validation_data=val_data,epochs=10,batch_size=32)

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/10
195/195 ━━━━━━━━━━━━━━━━━━━━ 127s 573ms/step - accuracy: 0.1410 - loss: 2.4073 - val_accuracy: 0.1712 - val_loss: 2.2944
Epoch 2/10
195/195 ━━━━━━━━━━━━━━━━━━━━ 98s 505ms/step - accuracy: 0.1764 - loss: 2.2877 - val_accuracy: 0.1712 - val_loss: 2.2738
Epoch 3/10
195/195 ━━━━━━━━━━━━━━━━━━━━ 96s 493ms/step - accuracy: 0.1726 - loss: 2.2612 - val_accuracy: 0.1712 - val_loss: 2.2384
Epoch 4/10
195/195 ━━━━━━━━━━━━━━━━━━━━ 142s 492ms/step - accuracy: 0.1808 - loss: 2.2340 - val_accuracy: 0.1905 - val_loss: 2.2094
Epoch 5/10
195/195 ━━━━━━━━━━━━━━━━━━━━ 142s 493ms/step - accuracy: 0.1919 - loss: 2.1984 - val_accuracy: 0.1658 - val_loss: 2.2016
Epoch 6/10
195/195 ━━━━━━━━━━━━━━━━━━━━ 98s 500ms/step - accuracy: 0.1966 - loss: 2.1776 - val_accuracy: 0.2262 - val_loss: 2.1660
Epoch 7/10
195/195 ━━━━━━━━━━━━━━━━━━━━ 97s 497ms/step - accuracy: 0.2034 - loss: 2.1684 - val_accuracy: 0.2271 - val_loss: 2.1329
Epoch 8/10
195/195 ━━━━━━━━━━

In [34]:
ResNet_loss,ResNet_accuracy=ResNet50_model.evaluate(test_data)

100/100 ━━━━━━━━━━━━━━━━━━━━ 14s 144ms/step - accuracy: 0.2642 - loss: 2.0855


In [35]:
ResNet50_base.trainable=True

ResNet50_model.compile(optimizer=Adam(learning_rate=0.0001),loss="categorical_crossentropy",metrics=["accuracy"])

ResNet50_model.fit(train_data,validation_data=val_data,epochs=5)

Epoch 1/5
195/195 ━━━━━━━━━━━━━━━━━━━━ 209s 752ms/step - accuracy: 0.5547 - loss: 4.1402 - val_accuracy: 0.1712 - val_loss: 34.1350
Epoch 2/5
195/195 ━━━━━━━━━━━━━━━━━━━━ 147s 613ms/step - accuracy: 0.9791 - loss: 0.1055 - val_accuracy: 0.1712 - val_loss: 34.7072
Epoch 3/5
195/195 ━━━━━━━━━━━━━━━━━━━━ 115s 588ms/step - accuracy: 0.9858 - loss: 0.0553 - val_accuracy: 0.1712 - val_loss: 30.7273
Epoch 4/5
195/195 ━━━━━━━━━━━━━━━━━━━━ 115s 590ms/step - accuracy: 0.9879 - loss: 0.0416 - val_accuracy: 0.1951 - val_loss: 17.1787
Epoch 5/5
195/195 ━━━━━━━━━━━━━━━━━━━━ 116s 596ms/step - accuracy: 0.9882 - loss: 0.0465 - val_accuracy: 0.8654 - val_loss: 0.5700


In [36]:
ResNet50_loss,ResNet50_accuracy=ResNet50_model.evaluate(test_data)

100/100 ━━━━━━━━━━━━━━━━━━━━ 12s 122ms/step - accuracy: 0.8726 - loss: 0.5777


In [52]:
ResNet50_model.save("ResNet50_fish_class.h5")
drive.mount('/content/drive')
!cp ResNet50_fish_class.h5 /content/drive/MyDrive/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [38]:
# Converting the dimensions to 224,224
train_data=train_gen.flow_from_directory(train,target_size=(224,224),class_mode="categorical")
test_data=val_gen.flow_from_directory(test,target_size=(224,224),class_mode="categorical")
val_data=val_gen.flow_from_directory(val,target_size=(224,224),class_mode="categorical")

Found 6225 images belonging to 11 classes.
Found 3187 images belonging to 11 classes.
Found 1092 images belonging to 11 classes.


## MobileNet Model

In [41]:
MobileNet_base=MobileNetV2(weights="imagenet",include_top=False,input_shape=(224,224,3))
MobileNet_model=create_transfer_model(MobileNet_base)

MobileNet_model.compile(optimizer=Adam(),loss="categorical_crossentropy",metrics=["accuracy"])

MobileNet_history=MobileNet_model.fit(train_data,validation_data=val_data,epochs=10,batch_size=64)

Epoch 1/10
195/195 ━━━━━━━━━━━━━━━━━━━━ 0s 403ms/step - accuracy: 0.7034 - loss: 0.9168

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


195/195 ━━━━━━━━━━━━━━━━━━━━ 95s 449ms/step - accuracy: 0.7041 - loss: 0.9145 - val_accuracy: 0.9560 - val_loss: 0.1290
Epoch 2/10
195/195 ━━━━━━━━━━━━━━━━━━━━ 76s 390ms/step - accuracy: 0.9587 - loss: 0.1412 - val_accuracy: 0.9753 - val_loss: 0.0753
Epoch 3/10
195/195 ━━━━━━━━━━━━━━━━━━━━ 77s 396ms/step - accuracy: 0.9738 - loss: 0.0910 - val_accuracy: 0.9844 - val_loss: 0.0515
Epoch 4/10
195/195 ━━━━━━━━━━━━━━━━━━━━ 78s 399ms/step - accuracy: 0.9779 - loss: 0.0671 - val_accuracy: 0.9826 - val_loss: 0.0535
Epoch 5/10
195/195 ━━━━━━━━━━━━━━━━━━━━ 80s 409ms/step - accuracy: 0.9851 - loss: 0.0499 - val_accuracy: 0.9872 - val_loss: 0.0302
Epoch 6/10
195/195 ━━━━━━━━━━━━━━━━━━━━ 79s 405ms/step - accuracy: 0.9854 - loss: 0.0487 - val_accuracy: 0.9881 - val_loss: 0.0370
Epoch 7/10
195/195 ━━━━━━━━━━━━━━━━━━━━ 78s 399ms/step - accuracy: 0.9864 - loss: 0.0399 - val_accuracy: 0.9853 - val_loss: 0.0393
Epoch 8/10
195/195 ━━━━━━━━━━━━━━━━━━━━ 83s 404ms/step - accuracy: 0.9832 - loss: 0.0510 - val

In [42]:
MobileNet_loss,MobileNet_accuracy=MobileNet_model.evaluate(test_data)

100/100 ━━━━━━━━━━━━━━━━━━━━ 9s 91ms/step - accuracy: 0.9905 - loss: 0.0305


In [43]:
MobileNet_base.trainable=True

MobileNet_model.compile(optimizer=Adam(learning_rate=0.0001),loss="categorical_crossentropy",metrics=["accuracy"])

MobileNet_model.fit(train_data,validation_data=val_data,epochs=5)

Epoch 1/5
195/195 ━━━━━━━━━━━━━━━━━━━━ 194s 570ms/step - accuracy: 0.8180 - loss: 1.8645 - val_accuracy: 0.8462 - val_loss: 0.6622
Epoch 2/5
195/195 ━━━━━━━━━━━━━━━━━━━━ 91s 432ms/step - accuracy: 0.9871 - loss: 0.0413 - val_accuracy: 0.9679 - val_loss: 0.1367
Epoch 3/5
195/195 ━━━━━━━━━━━━━━━━━━━━ 84s 431ms/step - accuracy: 0.9921 - loss: 0.0229 - val_accuracy: 0.9670 - val_loss: 0.0935
Epoch 4/5
195/195 ━━━━━━━━━━━━━━━━━━━━ 142s 429ms/step - accuracy: 0.9926 - loss: 0.0226 - val_accuracy: 0.9826 - val_loss: 0.0680
Epoch 5/5
195/195 ━━━━━━━━━━━━━━━━━━━━ 83s 425ms/step - accuracy: 0.9948 - loss: 0.0164 - val_accuracy: 0.9872 - val_loss: 0.0538


In [44]:
MobileNet_tuned_loss,MobileNet_tuned_accuracy=MobileNet_model.evaluate(test_data)

100/100 ━━━━━━━━━━━━━━━━━━━━ 8s 76ms/step - accuracy: 0.9802 - loss: 0.0623


In [45]:
MobileNet_model.save("MobileNet_fish_class.h5")
drive.mount('/content/drive')
!cp MobileNet_fish_class.h5 /content/drive/MyDrive/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## EfficientNetB0 model

In [46]:
EfficientNet_base=EfficientNetB0(weights="imagenet",include_top=False,input_shape=(224,224,3))
EfficientNet_model=create_transfer_model(EfficientNet_base)

EfficientNet_model.compile(optimizer=Adam(),loss="categorical_crossentropy",metrics=["accuracy"])

EfficientNet_history=EfficientNet_model.fit(train_data,validation_data=val_data,epochs=5,batch_size=64)

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/5
195/195 ━━━━━━━━━━━━━━━━━━━━ 124s 525ms/step - accuracy: 0.1533 - loss: 2.3539 - val_accuracy: 0.1712 - val_loss: 2.3166
Epoch 2/5
195/195 ━━━━━━━━━━━━━━━━━━━━ 108s 408ms/step - accuracy: 0.1664 - loss: 2.3209 - val_accuracy: 0.1712 - val_loss: 2.3119
Epoch 3/5
195/195 ━━━━━━━━━━━━━━━━━━━━ 79s 407ms/step - accuracy: 0.1681 - loss: 2.3152 - val_accuracy: 0.1712 - val_loss: 2.3139
Epoch 4/5
195/195 ━━━━━━━━━━━━━━━━━━━━ 78s 401ms/step - accuracy: 0.1687 - loss: 2.3144 - val_accuracy: 0.1712 - val_loss: 2.3134
Epoch 5/5
195/195 ━━━━━━━━━━━━━━━━━━━━ 82s 401ms/step - accuracy: 0.1774 - loss: 2.3049 - val_accuracy: 0.1712 - val_loss: 2.3168


In [47]:
EfficientNet_loss,EfficientNet_accuracy=EfficientNet_model.evaluate(test_data)

100/100 ━━━━━━━━━━━━━━━━━━━━ 11s 111ms/step - accuracy: 0.1620 - loss: 2.3018


In [48]:
EfficientNet_base.trainable=True

EfficientNet_model.compile(optimizer=Adam(),loss="categorical_crossentropy",metrics=["accuracy"])

EfficientNet_tuned_history=EfficientNet_model.fit(train_data,validation_data=val_data,epochs=5)

Epoch 1/5
195/195 ━━━━━━━━━━━━━━━━━━━━ 214s 678ms/step - accuracy: 0.7959 - loss: 0.6657 - val_accuracy: 0.1712 - val_loss: 2.6318
Epoch 2/5
195/195 ━━━━━━━━━━━━━━━━━━━━ 86s 438ms/step - accuracy: 0.9737 - loss: 0.0936 - val_accuracy: 0.1639 - val_loss: 5.3083
Epoch 3/5
195/195 ━━━━━━━━━━━━━━━━━━━━ 84s 432ms/step - accuracy: 0.9796 - loss: 0.0718 - val_accuracy: 0.0092 - val_loss: 4.0590
Epoch 4/5
195/195 ━━━━━━━━━━━━━━━━━━━━ 86s 439ms/step - accuracy: 0.9765 - loss: 0.0796 - val_accuracy: 0.1282 - val_loss: 54.3538
Epoch 5/5
195/195 ━━━━━━━━━━━━━━━━━━━━ 87s 446ms/step - accuracy: 0.9893 - loss: 0.0342 - val_accuracy: 0.1712 - val_loss: 21.8417


In [49]:
EfficientNet_tuned_loss,EfficientNet_tuned_accuracy=EfficientNet_model.evaluate(test_data)

100/100 ━━━━━━━━━━━━━━━━━━━━ 10s 102ms/step - accuracy: 0.1652 - loss: 21.9335


In [50]:
EfficientNet_model.save("EfficientNet_fish_class.h5")
drive.mount('/content/drive')
!cp EfficientNet_fish_class.h5 /content/drive/MyDrive/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [55]:
print(f'Base model accuracy: {base_accurracy:.3f}, loss: {base_loss:.3f}')
print('---------------------------------------------------')
print(f'VGG16 model accuracy: {VGG16_accuracy:.3f}, loss: {VGG16_loss:.3f}')
print('---------------------------------------------------')
print(f'ResNet50 model accuracy: {ResNet_accuracy:.3f}, loss: {ResNet_loss:.3f}')
print('---------------------------------------------------')
print(f'MobileNet model accuracy: {MobileNet_accuracy:.4f}, loss: {MobileNet_loss:.4f}')
print('---------------------------------------------------')
print(f'EfficientNetB0 model accuracy: {EfficientNet_accuracy:.4f}, loss: {EfficientNet_loss:.4f}')

Base model accuracy: 0.941, loss: 0.207
---------------------------------------------------
VGG16 model accuracy: 0.969, loss: 0.134
---------------------------------------------------
ResNet50 model accuracy: 0.252, loss: 2.096
---------------------------------------------------
MobileNet model accuracy: 0.9931, loss: 0.0213
---------------------------------------------------
EfficientNetB0 model accuracy: 0.1632, loss: 2.3053
